In [1]:
%use kandy, multik
import generator.Generator
import generator.Point

In [187]:
import java.text.DecimalFormat

val repeats = 10000
var blocks = 0
repeat(repeats) {
    val (p1, p2) = Generator.nextPariInSquare(h, A)
    val obstacles = HashSet<Point>()
    repeat(poissonDistribution.sample()) {
        obstacles.add(Generator.nextPointInSquare(A))
    }
    var block = false
    check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
        for (p in obstacles) {
            if (i.isInCircle(p, r)) {
                block = true
                break@check
            }
        }
    }
    if (block) {
        ++blocks
    }
}
println("Probability: ${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}")

Probability: 0.387


In [196]:
import org.apache.commons.math3.distribution.PoissonDistribution
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

val hs = listOf<Number>(0.1, 1, 4, 10) // point distance
val `as` = listOf<Number>(0.01, 0.05, 0.1, 0.3, 0.5) // obstacle density
val rs = listOf<Number>(0.05, 0.1, 1, 3) // obstacle size


println("a\th\tr\tProbability")
for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = false
                check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                    for (p in obstacles) {
                        if (i.isInCircle(p, r)) {
                            block = true
                            break@check
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            println("$a\t$h\t$r\t${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}")
        }
    }
}

a	h	r	Probability
0.01	0.1	0.05	0.00
0.01	0.1	0.1	0.00
0.01	0.1	1	0.031
0.01	0.1	3	0.208
0.01	1	0.05	0.00
0.01	1	0.1	0.002
0.01	1	1	0.041
0.01	1	3	0.231
0.01	4	0.05	0.003
0.01	4	0.1	0.006
0.01	4	1	0.096
0.01	4	3	0.377
0.01	10	0.05	0.01
0.01	10	0.1	0.02
0.01	10	1	0.195
0.01	10	3	0.462
0.05	0.1	0.05	0.00
0.05	0.1	0.1	0.003
0.05	0.1	1	0.165
0.05	0.1	3	0.656
0.05	1	0.05	0.004
0.05	1	0.1	0.011
0.05	1	1	0.235
0.05	1	3	0.71
0.05	4	0.05	0.015
0.05	4	0.1	0.045
0.05	4	1	0.407
0.05	4	3	0.894
0.05	10	0.05	0.039
0.05	10	0.1	0.105
0.05	10	1	0.66
0.05	10	3	0.951
0.1	0.1	0.05	0.001
0.1	0.1	0.1	0.01
0.1	0.1	1	0.256
0.1	0.1	3	0.866
0.1	1	0.05	0.006
0.1	1	0.1	0.026
0.1	1	1	0.371
0.1	1	3	0.907
0.1	4	0.05	0.044
0.1	4	0.1	0.075
0.1	4	1	0.661
0.1	4	3	0.991
0.1	10	0.05	0.111
0.1	10	0.1	0.174
0.1	10	1	0.897
0.1	10	3	0.999
0.3	0.1	0.05	0.007
0.3	0.1	0.1	0.011
0.3	0.1	1	0.606
0.3	0.1	3	0.997
0.3	1	0.05	0.031
0.3	1	0.1	0.068
0.3	1	1	0.745
0.3	1	3	0.999
0.3	4	0.05	0.106
0.3	4	0.1	0.215
0.3	4	1	0.966
0.3	4	3	1.00
0

In [214]:
import org.apache.commons.math3.distribution.PoissonDistribution

val A = 10 // square side

val h = 1 // point distance
val a = 0.1 // obstacle density
val r = 0.3 // obstacle size

val poissonDistribution = PoissonDistribution(a * A * A)

val (p1, p2) = Generator.nextPariInSquare(h, A)
val obstacles = HashSet<Point>()
repeat(poissonDistribution.sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

val psize = 25
plot {
    
    points { 
        x(obstacles.map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(obstacles.map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        size = psize.toDouble() * r
    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        type = LineType.DASHED
        color = Color.RED
    }

    layout {
        size = Pair(A * psize, A * psize)
        style { 
            axis {
                blankAxes()
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="b3j9pQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":250.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.350387563257286,6.096302500541321,1.0926554371650599,9.055118566153727,2.1818656242193377,6.804628553035297,5.024717128859306,6.811073143570087,4.218829501829326,0.2896607235726645,7.73302943410161,2.848427209715302],
"y":[7.465227078594135,5.01596508173029,2.8287960066465114,0.11275054380814997,0.3373646099401584,4.990178980212315,0.3864780850937821,4.318744100736556,5.928039795468438,7.344799156411402,3.5305908035132907,7.117550316686019]
},
"size":7.5,
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.319317406160088,4.680682593839912],
"y":[4.615245020664847,5.384754979335153]
},
"color":"#ee6666",
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.319317406160088,4.680682593839912],
"y":[4.615245020664847,5.384754979335153]
},
"color":"#ee6666",
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"line"
}],
"theme":{
"axis":"blank",
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("b3j9pQ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>">